In [2]:
import tensorflow as tf
import numpy as np

In [ ]:
#Load dataset
train_dataset=tf.keras.utils.image_dataset_from_directory(
                                                          directory="",
                                                          labels="inferred",
                                                          label_mode="int",
                                                          color_mode="rgb",
                                                          batch_size=None,
                                                          image_size=(256, 256),
                                                          shuffle=True,
                                                          interpolation="bilinear"
                                                          )

test_dataset=tf.keras.utils.image_dataset_from_directory(
                                                          directory="",
                                                          labels="inferred",
                                                          label_mode="int",
                                                          color_mode="rgb",
                                                          batch_size=None,
                                                          image_size=(256, 256),
                                                          shuffle=True,
                                                          interpolation="bilinear"
                                                          )

In [ ]:
# count=0
# images_data=[]
# label_data=[]
# for images, labels in train_ds:
#   for img,label in zip(images,labels):
#       print(img.shape,label)
#       images_data.append(img)
#       label_data.append(label)

images_data=[]
label_data=[]
for images, labels in train_dataset:
  images_data.append(images)
  label_data.append(labels.numpy())
  print(count,">>>>",images.shape,">>",labels.numpy())
  count=count+1

In [36]:
#https://www.tensorflow.org/guide/keras/preprocessing_layers
#preprocessing layer
data_augmentation = tf.keras.Sequential(
    
                                        [   
                                         tf.keras.layers.Resizing(height=64,width=64,interpolation="bilinear", crop_to_aspect_ratio=False),
                                         tf.keras.layers.Rescaling(scale=1./255, offset=0.0),
                                         tf.keras.layers.RandomFlip(mode="horizontal_and_vertical"),
                                         tf.keras.layers.RandomRotation(factor=(-.2,0.3),fill_mode="nearest",interpolation="bilinear",fill_value="constant"),
                                         tf.keras.layers.RandomZoom(height_factor=(0.2, 0.3),width_factor=(-0.3, -0.2),fill_mode="reflect",interpolation="bilinear",fill_value="constant"),
                                         tf.keras.layers.RandomRotation(0.1),
                                         tf.keras.layers.RandomZoom(0.1),
                                        ]

                                       )


#convert in the tensor preprocess it make batches example is below
"""
(x_train, y_train), _ = tf.keras.datasets.cifar10.load_data()
input_shape = x_train.shape[1:]
classes = 10

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.batch(16).map(lambda x, y: (data_augmentation(x), y))

"""



(x_train, y_train), _ = tf.keras.datasets.cifar10.load_data()
input_shape = x_train.shape[1:]
classes = 10

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.batch(64).map(lambda x, y: (data_augmentation(x), y))

In [16]:
#restnet 50
image_input = tf.keras.layers.Input(shape=(64,64, 3)) 
model_pretrained=tf.keras.applications.resnet50.ResNet50(
                                                         include_top=False,
                                                         weights='imagenet',
                                                         input_tensor=image_input,
                                                         pooling=max,
                                                         )
for i,layer in enumerate(model_pretrained.layers):  
  layer.trainable=False


model_pretrained.summary()


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_11[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [19]:
#custom dense layer

FC_layer_Flatten = tf.keras.layers.Flatten()(model_pretrained.output)    
Dense=tf.keras.layers.Dense(units=1000,activation='relu')(FC_layer_Flatten)    
Dense=tf.keras.layers.Dense(units=800,activation='relu')(Dense)
Dense=tf.keras.layers.Dense(units=400,activation='relu')(Dense)
Dense=tf.keras.layers.Dense(units=200,activation='relu')(Dense)
Dense=tf.keras.layers.Dense(units=100,activation='relu')(Dense)
Classification=tf.keras.layers.Dense(units=6,activation='softmax')(Dense)


model_final = tf.keras.Model(inputs=image_input,outputs=Classification)
model_final.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_11[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            